# Высокие данные

Опять будем решать ту же самую задачу - предсказывать цену по описанию. Однако теперь данных в 2 раза больше - будем считать, что это уже много и пытаться решить эту проблему различными способами.

# Распределенный градиентный спуск

Классический алгоритм для оптимизации функции потерь - градиентный спуск. Чтобы посчитать градиентный спуск необходимо подсчитать градиент на каждом объекте выборки и сложить. При этом подсчет градиента на каждом объекта - это независимая операция.

Уже видно из описания, что это процесс, который достаточно хорошо ложится на парадигму вычислений в Spark. Рабочие узлы могут параллельно подсчитать градиент, после чего его получит мастер-машина (клиент, который управляет вычислениями), сделает шаг в направлении и разошлет новые веса рабочим. Эти шаги нужно будет повторять, пока градиентный спуск не сойдется.

По сути это очень простая версия подхода "Сервер параметров (Parameter server)". И она уже реализована в Spark ML.

In [1]:
import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import SparkSession


conf = pyspark.SparkConf().setAppName("CourseraLocalSpark").setMaster("local[*]")
sc = pyspark.SparkContext.getOrCreate(conf)

Собираем набор данных

In [2]:
spark = SparkSession.builder.appName('DIstributed-ML').getOrCreate()
airbnb = spark.read.option("header",True).option("delimiter", "\t").csv('airbnb-200k.tsv')
airbnb.createOrReplaceTempView("airbnb")

raw_df = spark.sql("""
    SELECT double(Price), split(Description, ' ') as Words
    FROM airbnb
    WHERE Description is not Null AND double(Price) is not Null
""")

Кодируем текст через Bag-Of-Words и сразу делим на обучающую и тестовую выборку

In [3]:
from pyspark.ml.feature import CountVectorizer


cv = CountVectorizer(inputCol="Words", outputCol="Features")
spark_vectorizer = cv.fit(raw_df)
X = spark_vectorizer.transform(raw_df)
X_train, X_test = X.randomSplit([0.7, 0.3], seed=100)

In [4]:
X_train.show()

+-----+--------------------+--------------------+
|Price|               Words|            Features|
+-----+--------------------+--------------------+
|  0.0|["Spacious, 19th,...|(262144,[0,1,2,3,...|
|  0.0|["The, accommodat...|(262144,[0,1,2,3,...|
|  0.0|[A, bright, chic,...|(262144,[0,1,2,3,...|
|  0.0|[A, great, safe, ...|(262144,[0,1,2,3,...|
|  0.0|[Beautiful, &, Br...|(262144,[0,1,2,3,...|
|  0.0|[Bedroom, in, 1st...|(262144,[0,1,2,3,...|
|  0.0|[Charming, and, c...|(262144,[0,1,2,3,...|
|  0.0|[Cute, exposed, b...|(262144,[0,1,2,3,...|
|  0.0|[Die, Wohnung, li...|(262144,[5,235,50...|
|  0.0|[I, have, a, priv...|(262144,[0,1,2,4,...|
|  0.0|[I, have, a, priv...|(262144,[0,1,2,4,...|
|  0.0|[Located, on, one...|(262144,[0,1,2,3,...|
|  0.0|[Marvelous, New, ...|(262144,[0,1,2,3,...|
|  0.0|[Minimalist, loft...|(262144,[0,1,2,3,...|
|  0.0|[Private, bedroom...|(262144,[0,1,2,3,...|
|  0.0|[Private,, separa...|(262144,[0,1,2,3,...|
|  0.0|[Stay, at, your, ...|(262144,[0,1,2,3,...|


Создаем объект линейной регрессии. Указываем, что признаки лежат в колонке Features, а целевая переменная - в Price.

In [5]:
from pyspark.ml.regression import LinearRegression

regressor = LinearRegression(featuresCol='Features', labelCol="Price", maxIter=10, regParam=0.3)

Запускаем обучение

In [6]:
%%time

regressor_model = regressor.fit(X_train)

CPU times: user 15.3 ms, sys: 0 ns, total: 15.3 ms
Wall time: 1min 19s


Считаем метрику качества на тестовой выборке

In [7]:
predictions = regressor_model.transform(X_test)
predictions.show(5)

+-----+--------------------+--------------------+-------------------+
|Price|               Words|            Features|         prediction|
+-----+--------------------+--------------------+-------------------+
|  0.0|[A, comfortable,,...|(262144,[0,1,2,3,...|-36.064333580915275|
|  0.0|[Die, Wohnung, li...|(262144,[5,89,151...| 54.743880511540596|
|  0.0|[I, am, renting, ...|(262144,[0,1,2,3,...| 151.95972634910615|
|  0.0|[It, is, a, cozy,...|(262144,[0,1,2,3,...|  47.71575250039061|
|  0.0|[It, is, a, new, ...|(262144,[0,1,2,4,...| 140.78350583790393|
+-----+--------------------+--------------------+-------------------+
only showing top 5 rows



In [8]:
from pyspark.ml.evaluation import RegressionEvaluator


lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Price", metricName="r2")
lr_evaluator.evaluate(predictions)

0.3197710094091262

Что ж, вполне неплохой результат, учитывая, что мы ничего не делали с данными и никак не настраивали дополнительно модель. Это решение может теоретически масштабироваться до того момента, пока данные вообще помещаются на кластере (а это много).

Можно ли лучше? 

# Distributed Vowpal Wabbit и Tree Allreduce

Уже известный нам инструмент VW позволяет решать задачу не только широких данных, но и высоких через подход Tree Allreduce. 

По сути своей этот подход похож на то, что мы делали с помощью спарка, однако теперь все вычислительные узлы собраны в дерево, что позволяет тратить гораздо меньше времени на сетевое взаимодействие между узлами.

Для работы с VW есть специальная утилита `spanning_tree` - это "вершина" дерева и менеджер всего процесса обучения. VW подключается по сети к этому сервису, получает свое место в дереве и начинает считать и обмениваться градиентами согласно схеме. 

По окончанию обучения, когда все узлы завершат свою работу, один из рабочих может сохранить полученные коэффициенты.

Перед тем, как обучать модель, преобразуем данные в формат vw.

In [9]:
df = pd.read_csv("airbnb-200k.tsv", delimiter='\t')

In [10]:
import re

def convert_to_vw(raw_text, target):
    word_pattern = re.compile(r"[a-zA-Z0-9_]+")
    words = []
    for match in re.finditer(word_pattern, raw_text.lower()):
        words.append(match.group(0))
    
    if not words: 
        return None
    return "{} |d {}".format(float(target), " ".join(words))

In [11]:
df.dropna(subset=['Price', "Description"], inplace=True)
Y = df['Price']
X = df['Description']

In [12]:
def write_vw(X_data, Y_data, filename):
    with open(filename, "w") as f:
        for x, y in zip(X_data, Y_data):
            vw_object = convert_to_vw(x, y)
            if not vw_object:
                continue
            f.write(vw_object + '\n')

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=100)

Далее нам нужно руками разделить данные на отдельные части. Каждую часть мы выдадим отдельному рабочему для обработки.

Тут важно также понимать, что в реальной жизни мы бы разделили данные сразу между несколькими машинами. И тогда бы данные равномерно распределились и каждый рабочий бы работал параллельно с другими на своей машине.

В нашем же окружении мы делаем все на одной машине для наглядности, но нужно держать в голове, что и `spanning_tree` и `vw` для максимально эффективности запускались бы на разным машинах.

In [15]:
X_train1, X_train2, y_train1, y_train2 = train_test_split(X_train, y_train, test_size=0.5, random_state=100)

In [16]:
write_vw(X_train1, y_train1, "airbnb-train-part1.vw")
write_vw(X_train2, y_train2, "airbnb-train-part2.vw")
write_vw(X_test, y_test, "airbnb-test.vw")

In [17]:
! head -n 1 airbnb-train-part1.vw

395.0 |d sunny unique home with a treehouse theme in the center of the bay area s attractions a 15 minute walk get you to the attractions of the castro mission district and soma with public transportation you can quickly get everywhere else if you find ladders levels wood and little nooks fun you are going to love the mission treehouse we treat everyone in the airbnb community regardless of their race religion national origin ethnicity disability sex gender identity sexual orientation or age with respect kindness and compassion and without judgment or bias this is an lgbtqia friendly household about the mission treehouse 2 bedroom and 2 full bathrooms excellent location the mission district near downtown moscone castro soma and the sights parking private attached garage full kitchen wonderful details and decor i have lovingly crafted my 3 story modern condo into a romantic playground of my dreams and i m happy to share it with you both bedrooms h


In [18]:
! head -n 1 airbnb-test.vw

50.0 |d un precioso piso con cocina y ba o reformados y agradable ambiente habitaci n principal muy amplia con cama matrimonio de 1 60cm sal n con comedor y 2 sof s uno de ellos convertible en cama una habitaci n individual con una cama peque a de 90cm cuarto piso exterior y muy luminoso a solo 20 minutos en autob s del centro de madrid puerta del sol estaci n de metro alto de extremadura l nea 6 en la misma plaza donde se encuentra la vivienda a solo 10 minutos en metro de plaza de espa a y a 5 minutos de pr ncipe p o centro comercial y de ocio shopping cines restaurantes rodeado de zonas verdes casa de campo y lago paseo de avenida de portugal y madrid r o todo el piso estar disponible para cualquier duda o ayuda que pod is necesitar avisando con antelaci n m vil phone number hidden barrio popular con muchos servicios en la misma plaza supermercado d a fruter as bares y cafeter as entrada al metro de madrid a unos pasos del portal de la vivienda varias l neas de a


Запускаем spanning_tree. Так как мы запускаем его из Jupyter notebook, то есть небольшая особенность - чтобы он правильно запустился в фоне, мы используем команду %%bash и параметром --bg (то есть запуск в фоне). Сам spanning_tree запускаем с --nondaemon потому что сама ячейка благодаря %%bash уже запустится в фоне.

На реально машине вы бы использовали просто команду `spanning_tree`.

In [19]:
%%bash --bg --out OUT --err ERR
spanning_tree --nondaemon

Проверяем, что он запустился

In [20]:
! ps aux | grep spanning_tree

jovyan     447  0.0  0.0   6068  1632 ?        S    21:10   0:00 spanning_tree --nondaemon
jovyan     448  0.0  0.0   6896  3276 pts/0    Ss+  21:10   0:00 /bin/bash -c  ps aux | grep spanning_tree
jovyan     450  0.0  0.0   6436   660 pts/0    S+   21:10   0:00 grep spanning_tree


Видим работающий процесс, значит все хорошо. 

Теперь давайте запускать рабочих. Для этого используется уже известная команда `vw`, в которую просто добавляются специальные параметры

* `--span_server` - указываем адрес, где находится менеджер (spanning_tree). В нашем случае это localhost. В реальной жизни там мог бы быть IP адрес другой машины
* `--unique_id` - так как один spanning_tree может обрабатывать сразу много различных процессов обучения, то необходимо их как-то разграничить. Для этого используется unique_id - это число, которое должно быть одинаковым для всех ваших рабочих, чтобы их не перепутали с другими. Например ваш коллега также обучает VW но для другой задачи - он может подключить свои VW к этому же spanning_tree указав для них unique_id = 0. В таком случае вам, чтобы подключиться, нужно запускать свои рабочие например с unique_id = 5, чтобы они не смешались с рабочими вашего коллеги.
* `--total` - число рабочих, которое вы планируете подключить в текущей сессии обучения
* `--node` - идентификатор текущего рабочего. Нумерация начинается с нуля, поэтому если вы хотите запустить 3 рабочих, то им нужно выдать значения для `--node` 0, 1 и 2.
* `-d` - данные для обработки для текущего рабочего

Все остальные параметры обучения должны быть одинаковыми для всех рабочих.

Чтобы сохранить коэффициенты полученной модели, необходимо для какого-то одного рабочего указать через `-f` или `--final_regressor` файл, куда записать результат. Точно также, как мы это делали в предыдущей лабораторной.

Запустим двух рабочих. Первого запустим также в фоне, а вот второй запустим прямо в ноутбуке и будем следить за процессом обучения.

In [21]:
%%bash --bg --out OUT --err ERR
vw --span_server localhost --total 2 --node 0 --unique_id 1 -d airbnb-train-part1.vw --learning_rate 10.0 --bit_precision 22 --passes 20 --cache_file vw.cache1

In [22]:
%%time
! vw --span_server localhost --total 2 --node 1 --unique_id 1 -d airbnb-train-part2.vw -f result.vw.bin --learning_rate 10.0 --bit_precision 22 --passes 20 --cache_file vw.cache2

final_regressor = result.vw.bin
Num weight bits = 22
learning rate = 10
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = vw.cache2
Reading datafile = airbnb-train-part2.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
2500.000000 2500.000000            1            1.0  50.0000   0.0000      139
1627.909149 755.818298            2            2.0  75.0000  47.5079      178
2737.117977 3846.326805            4            4.0 100.0000  13.5438       36
2509.605871 2282.093765            8            8.0  90.0000 115.6549      172
3019.351312 3529.096753           16           16.0 125.0000  42.2387      131
24120.006490 45220.661667           32           32.0  18.0000  12.4142       57
55243.550235 86367.093980           64           64.0  84.0000 346.3909      187
39819.820698 24396.091162          128          128.0 

Посчитаем качество

In [23]:
from sklearn.metrics import r2_score


def read_target_from_vw(vw_object):
    return float(vw_object.split(' ')[0])

def calc_r2(predictions_path, answers_path):
    with open(predictions_path, 'r') as f:
        y_pred = np.array([float(value) for value in f.readlines()])
        
    with open(answers_path, 'r') as f:
        y_expected = np.array([read_target_from_vw(value) for value in f.readlines()])
        
    return r2_score(y_expected, y_pred)

In [24]:
! vw --testonly --initial_regressor result.vw.bin --predictions predictions.txt airbnb-test.vw

only testing
predictions = predictions.txt
Num weight bits = 22
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = airbnb-test.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
2500.000000 2500.000000            1            1.0  50.0000   0.0000      193
1361.884918 223.769836            2            2.0  77.0000  91.9589      168
13769.519379 26177.153839            4            4.0 320.0000  92.7128      171
8775.625010 3781.730640            8            8.0  75.0000  61.4763      197
6514.805589 4253.986169           16           16.0  50.0000  71.4327       59
7108.112527 7701.419464           32           32.0  39.0000  48.9865      185
17976.902963 28845.693399           64           64.0 249.0000 105.9728      152
14032.351608 10087.800252          128          128.0 201.0000 146.9716       37
13260.489

In [25]:
calc_r2("predictions.txt", "airbnb-test.vw")

0.4728293051772131

В текущем окружении сильно большой разницы в скорости обучения мы не заметим, потому как 
* данных на самом деле очень немного
* мы все еще запускаем все на одной машине

Однако можно видеть, что концептуально схема совершенно рабочая - только что мы распределенно рассчитали линейную модель и получили относительно неплохое качество.

Более того можно видеть, что VW за счет своих оптимизаций (хеширование, tree allreduce и так далее) значительно обходит решение на Spark как по скорости расчета, так и по качеству получаемой модели